In [1]:
import pandas as pd
import os
from unicodedata import normalize
import re
import math as m
import datetime as dt
import numpy as np

# Init

In [3]:
path01 = 'your_path1'
path02 = 'your_path2'
path03 = 'your_path3'

In [4]:
def clean_datalist(data):
    #Se toman todos los tweets y se pasan a lista
    #data = dft.text.values.tolist()
    #Remover e-mails y menciones de otras cuentas de twitter con @
    data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]
    #Remover caracteres de nueva línea
    data = [re.sub('\s+', ' ', sent) for sent in data]
    #Remover comillas simples
    data = [re.sub("\'", "", sent) for sent in data]
    #Remover URLs hhtp y https
    data = [re.sub(r"https://\S+", "", sent) for sent in data]
    data = [re.sub(r"http://\S+", "", sent) for sent in data]
    #Remover datos entre corchetes y parentesis por ejemplo [VIDEO]
    data = [re.sub("[\(\[].*?[\)\]]", "", sent) for sent in data]
    #Remove Minusculas
    data = [sent.lower() for sent in data]
    #Maneja ñ
    data = [sent.replace('ñ','n') for sent in data]
    #Remover puntuaciones
    data = [re.sub('[^\w\s]', "", sent) for sent in data]
    #Remover acentos
    data = [normalize('NFC',re.sub(r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", normalize( "NFD", sent), 0, re.I)) for sent in data]
    #Remover espacios de inicio y fin
    data = [sent.lstrip(' ').rstrip(' ') for sent in data]
    
    return data

In [5]:
dict_regiones = {
    'tarapaca' : 1,
    'antofagasta' : 2,
    'atacama' : 3,
    'coquimbo' : 4,
    'valparaiso' : 5,
    'ohiggins' : 6,
    'maule' : 7,
    'biobio' : 8,
    'araucania' : 9,
    'los lagos' : 10,
    'aysen' : 11,
    'magallanes' : 12,
    'metropolitana' : 13,
    'los rios' : 14,
    'arica y parinacota' : 15,
    'nuble' : 16
}

dict_regiones2 = {
    1:'tarapaca',
    2:'antofagasta',
    3:'atacama',
    4:'coquimbo',
    5:'valparaiso',
    6:'ohiggins',
    7:'maule',
    8:'biobio',
    9:'araucania',
    10:'los lagos',
    11:'aysen',
    12:'magallanes',
    13:'metropolitana',
    14:'los rios',
    15:'arica y parinacota',
    16:'nuble'
}

# R Values

In [7]:
files_r = os.listdir(path_R)

dfr = pd.read_csv(path_R+files_r[0],index_col=0)
dfr['region'] = clean_datalist([files_r[0].split('_')[0]])[0]

for i,j in enumerate(files_r[1:]):
    df_aux = pd.read_csv(path_R+j,index_col=0)
    df_aux['region'] = clean_datalist([j.split('_')[0]])[0]
    dfr = dfr.append(df_aux)

dfr = dfr.reset_index(drop=True)
dfr['cod_reg'] = dfr.region.map(dict_regiones)
dfr['cod_reg'] = dfr['cod_reg'].apply(lambda x: 0 if np.isnan(x) else int(x))
dfr

,t_start,t_end,Mean(R),Std(R),Quantile.0.025(R),Quantile.0.05(R),Quantile.0.25(R),Median(R),Quantile.0.75(R),Quantile.0.95(R),Quantile.0.975(R),fecha_casos,region,cod_reg
0,2,8,5.000000,5.000000,0.126589,0.256466,1.438410,3.465736,6.931472,14.978661,18.444397,2020-03-03,antofagasta,2
1,3,9,5.000000,5.000000,0.126589,0.256466,1.438410,3.465736,6.931472,14.978661,18.444397,2020-03-04,antofagasta,2
2,4,10,5.000000,5.000000,0.126589,0.256466,1.438410,3.465736,6.931472,14.978661,18.444397,2020-03-05,antofagasta,2
3,5,11,5.000000,5.000000,0.126589,0.256466,1.438410,3.465736,6.931472,14.978661,18.444397,2020-03-06,antofagasta,2
4,6,12,15.000000,8.660254,3.093361,4.088457,8.636497,13.370302,19.602010,31.478968,36.123438,2020-03-07,antofagasta,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9005,527,533,0.835557,0.041935,0.755376,0.767796,0.806902,0.834855,0.863446,0.905710,0.919723,2021-08-10,biobio,8
9006,528,534,0.865269,0.043318,0.782440,0.795272,0.835671,0.864546,0.894079,0.937732,0.952205,2021-08-11,biobio,8
9007,529,535,0.943123,0.045748,0.855578,0.869157,0.911877,0.942384,0.973564,1.019613,1.034871,2021-08-12,biobio,8
9008,530,536,0.977558,0.046763,0.888041,0.901932,0.945623,0.976812,1.008679,1.055726,1.071311,2021-08-13,biobio,8


# Build Imported Cases

In [8]:
dfr_s2_all = dfr[(dfr.fecha_casos >= '2020-05-08') & (dfr.fecha_casos <= '2020-05-17')].copy()
dfr_s3_all = dfr[(dfr.fecha_casos >= '2020-05-23') & (dfr.fecha_casos <= '2020-06-01')].copy()

dfr_s2 = dfr_s2_all[dfr_s2_all.fecha_casos==dfr_s2_all.fecha_casos.max()].copy().reset_index(drop=True)
dfr_s3 = dfr_s3_all[dfr_s3_all.fecha_casos==dfr_s3_all.fecha_casos.max()].copy().reset_index(drop=True)

In [ ]:
dfic_s2 = pd.read_csv(path02+'imported_cases_and_incidence_together_s2.csv')
dfic_s2

In [ ]:
dfic_s2 = pd.read_csv(path02+'imported_cases_and_incidence_together_s2.csv')

dfic_s2 = dfic_s2.merge(dfr_s2[['Median(R)','Quantile.0.025(R)','Quantile.0.975(R)','cod_reg']],how='left',left_on='region_destination_code',right_on='cod_reg')
dfic_s2 = dfic_s2.rename(columns={'Median(R)':'r_mid', 'Quantile.0.025(R)':'r_low', 'Quantile.0.975(R)':'r_high'})
dfic_s2.drop(columns=['cod_reg'],inplace=True)

dfic_s2['expected_imported_cases_scenario_2'] = dfic_s2.expected_imported_cases_scenario_2_mid.round(2).astype(str)+' ('+dfic_s2.expected_imported_cases_scenario_2_low.round(2).astype(str)+', '+\
                                        dfic_s2.expected_imported_cases_scenario_2_high.round(2).astype(str)+')'

dfic_s2['expected_imported_cases_scenario_2sim'] = dfic_s2.expected_imported_cases_scenario_2sim_mid.round(2).astype(str)+' ('+dfic_s2.expected_imported_cases_scenario_2sim_low.round(2).astype(str)+', '+\
                                        dfic_s2.expected_imported_cases_scenario_2sim_high.round(2).astype(str)+')'

dfic_s2['local_cases_scenario_2'] = dfic_s2.new_cases_adjusted_mean_mid.round(2).astype(str)+' ('+dfic_s2.new_cases_adjusted_mean_low.round(2).astype(str)+', '+\
                                        dfic_s2.new_cases_adjusted_mean_high.round(2).astype(str)+')'

dfic_s2['risk_rating_scenario_2'] = (dfic_s2.expected_imported_cases_scenario_2_mid/dfic_s2.new_cases_adjusted_mean_mid).round(3).astype(str)+' ('+\
                                        (dfic_s2.expected_imported_cases_scenario_2_low/dfic_s2.new_cases_adjusted_mean_low).round(3).astype(str)+', '+\
                                        (dfic_s2.expected_imported_cases_scenario_2_high/dfic_s2.new_cases_adjusted_mean_high).round(3).astype(str)+')'

dfic_s2['risk_rating_scenario_2sim'] = (dfic_s2.expected_imported_cases_scenario_2sim_mid/dfic_s2.new_cases_adjusted_mean_mid).round(3).astype(str)+' ('+\
                                        (dfic_s2.expected_imported_cases_scenario_2sim_low/dfic_s2.new_cases_adjusted_mean_low).round(3).astype(str)+', '+\
                                        (dfic_s2.expected_imported_cases_scenario_2sim_high/dfic_s2.new_cases_adjusted_mean_high).round(3).astype(str)+')'

dfic_s2['rt_estimate_scenario_2'] = dfic_s2.r_mid.round(2).astype(str)+' ('+dfic_s2.r_low.round(2).astype(str)+', '+\
                                        dfic_s2.r_high.round(2).astype(str)+')'

dfic_s2['region'] = dfic_s2.region_destination_code.map(dict_regiones2)
dfic_s2 = dfic_s2.rename(columns={'region_destination_code':'cod_reg'})

dfic_s2.to_csv(path03+'generated/results_s2.csv')

dfic_s2[['cod_reg','region','expected_imported_cases_scenario_2','risk_rating_scenario_2','local_cases_scenario_2','rt_estimate_scenario_2',
         'risk_rating_scenario_2sim','expected_imported_cases_scenario_2sim']]

In [ ]:
dfic_s3 = pd.read_csv(path02+'imported_cases_and_incidence_together_s3.csv')

dfic_s3 = dfic_s3.merge(dfr_s3[['Median(R)','Quantile.0.025(R)','Quantile.0.975(R)','cod_reg']],how='left',left_on='region_destination_code',right_on='cod_reg')
dfic_s3 = dfic_s3.rename(columns={'Median(R)':'r_mid', 'Quantile.0.025(R)':'r_low', 'Quantile.0.975(R)':'r_high'})
dfic_s3.drop(columns=['cod_reg'],inplace=True)

dfic_s3['expected_imported_cases_scenario_3'] = dfic_s3.expected_imported_cases_scenario_3_mid.round(2).astype(str)+' ('+dfic_s3.expected_imported_cases_scenario_3_low.round(2).astype(str)+', '+\
                                        dfic_s3.expected_imported_cases_scenario_3_high.round(2).astype(str)+')'

dfic_s3['expected_imported_cases_scenario_3sim'] = dfic_s3.expected_imported_cases_scenario_3sim_mid.round(2).astype(str)+' ('+dfic_s3.expected_imported_cases_scenario_3sim_low.round(2).astype(str)+', '+\
                                        dfic_s3.expected_imported_cases_scenario_3sim_high.round(2).astype(str)+')'

dfic_s3['local_cases_scenario_3'] = dfic_s3.new_cases_adjusted_mean_mid.round(2).astype(str)+' ('+dfic_s3.new_cases_adjusted_mean_low.round(2).astype(str)+', '+\
                                        dfic_s3.new_cases_adjusted_mean_high.round(2).astype(str)+')'

dfic_s3['risk_rating_scenario_3'] = (dfic_s3.expected_imported_cases_scenario_3_mid/dfic_s3.new_cases_adjusted_mean_mid).round(3).astype(str)+' ('+\
                                        (dfic_s3.expected_imported_cases_scenario_3_low/dfic_s3.new_cases_adjusted_mean_low).round(3).astype(str)+', '+\
                                        (dfic_s3.expected_imported_cases_scenario_3_high/dfic_s3.new_cases_adjusted_mean_high).round(3).astype(str)+')'

dfic_s3['risk_rating_scenario_3sim'] = (dfic_s3.expected_imported_cases_scenario_3sim_mid/dfic_s3.new_cases_adjusted_mean_mid).round(3).astype(str)+' ('+\
                                        (dfic_s3.expected_imported_cases_scenario_3sim_low/dfic_s3.new_cases_adjusted_mean_low).round(3).astype(str)+', '+\
                                        (dfic_s3.expected_imported_cases_scenario_3sim_high/dfic_s3.new_cases_adjusted_mean_high).round(3).astype(str)+')'

dfic_s3['rt_estimate_scenario_3'] = dfic_s3.r_mid.round(2).astype(str)+' ('+dfic_s3.r_low.round(2).astype(str)+', '+\
                                        dfic_s3.r_high.round(2).astype(str)+')'

dfic_s3['region'] = dfic_s3.region_destination_code.map(dict_regiones2)
dfic_s3 = dfic_s3.rename(columns={'region_destination_code':'cod_reg'})

dfic_s3.to_csv(path03+'generated/results_s3.csv')

dfic_s3[['cod_reg','region','expected_imported_cases_scenario_3','risk_rating_scenario_3','local_cases_scenario_3','rt_estimate_scenario_3',
         'risk_rating_scenario_3sim','expected_imported_cases_scenario_3sim']]